In [1]:
import torchvision
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
# data set, data loader

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# model

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.fc1(torch.flatten(x, 1))
        x = self.fc2(torch.relu(x))
        
        return x
    
# training, test function

def train_eval(model):
    criterion = nn.CrossEntropyLoss()
    
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Training
    
    print("Training......................................")
    
    for epoch in range(5):
        running_loss = 0.0
        
        num_batches = 0
        
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            num_batches += 1 
            
            if (i + 1) % 200 == 190:
                print("Epoch # %d, Batch # %d, Loss: %.3f" % (epoch + 1, i + 1, running_loss / num_batches)) ####
                
                running_loss = 0.0
                
                num_batches = 0
            """
            한 epoch 내에서, 매 batch마다 loss를 계속 누적 계산하여 running_loss에 저장하다가, 
            왜인진 모르겠으나... 190, 390, 590, ...번째 batch일 때, running_loss를 초기화.
            
            현재의 running_loss가 몇 batch동안에 쌓여 만들어진 건지를, num_batches 통해 나타냄.
            
            결국, running_loss / num_batches => 이 식은, 
            num_batches만큼의 batch 동안의 loss의 "평균"을 나타냄.
            """
    
    # Test
    
    print("Test..........................................")
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * (correct / total)
    
    print("Test Accuracy: %.2f%%" % acc)
    
#

model = LeNet()

train_eval(model)

print("\n전체 파라미터 개수:\n", sum(p.numel() for p in model.parameters()))

Files already downloaded and verified
Files already downloaded and verified
Training......................................
Epoch # 1, Batch # 190, Loss: 2.258
Epoch # 1, Batch # 390, Loss: 2.064
Epoch # 1, Batch # 590, Loss: 1.923
Epoch # 2, Batch # 190, Loss: 1.732
Epoch # 2, Batch # 390, Loss: 1.669
Epoch # 2, Batch # 590, Loss: 1.611
Epoch # 3, Batch # 190, Loss: 1.558
Epoch # 3, Batch # 390, Loss: 1.517
Epoch # 3, Batch # 590, Loss: 1.494
Epoch # 4, Batch # 190, Loss: 1.448
Epoch # 4, Batch # 390, Loss: 1.425
Epoch # 4, Batch # 590, Loss: 1.410
Epoch # 5, Batch # 190, Loss: 1.356
Epoch # 5, Batch # 390, Loss: 1.346
Epoch # 5, Batch # 590, Loss: 1.325
Test..........................................
Test Accuracy: 56.30%

전체 파라미터 개수:
 167562
